# Project 2

We're going to continue from where we left off with Project 1. Project 1 left us with a daily time series for every product with no gaps -- exactly what we want for modeling!

In [1]:
data_path = 'data'

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_parquet(f'{data_path}/sales_data.parquet')
data["item_id"] = data["item_id"].astype("category")
data["dept_id"] = data["dept_id"].astype("category")
data["cat_id"] = data["cat_id"].astype("category")
data["store_id"] = data["store_id"].astype("category")
data["state_id"] = data["state_id"].astype("category")
data.head()

item_id  dept_id cat_id store_id  \
date       id                                                                  
2013-01-01 FOODS_1_004_TX_1_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_1   
           FOODS_1_004_TX_2_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_2   
           FOODS_1_004_TX_3_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_3   
           FOODS_1_005_TX_2_evaluation  FOODS_1_005  FOODS_1  FOODS     TX_2   
           FOODS_1_009_TX_2_evaluation  FOODS_1_009  FOODS_1  FOODS     TX_2   

                                       state_id  sales  
date       id                                           
2013-01-01 FOODS_1_004_TX_1_evaluation       TX     20  
           FOODS_1_004_TX_2_evaluation       TX     20  
           FOODS_1_004_TX_3_evaluation       TX      4  
           FOODS_1_005_TX_2_evaluation       TX      1  
           FOODS_1_009_TX_2_evaluation       TX      3

In [172]:
data.shape

(11322748, 6)

We did some EDA in Project 1, but it was primarily focused on higher level patterns (i.e., at the department level). This time, spend some time doing EDA at the item level to see what kind of items you're dealing with.

Some questions you may want to explore:

1. How do high-volume items compare to low-volume/itermittent items?
2. What sort of seasonal patterns are at play?
3. Do items from different departments show different patterns?
4. Does the same item show different behavior at different stores?

These questions are just a starting point. Feel free to explore this any way you feel is necessary to make better models. The best EDA is done iteratively, so I encourage you to come back to this once you've started fitting models!

In [173]:
# do some EDA!

data.groupby('dept_id', observed=True)['sales'].sum().nlargest(10)

dept_id
FOODS_3        6249806
HOUSEHOLD_1    2636374
FOODS_2        1399522
HOBBIES_1      1005896
FOODS_1         911553
HOUSEHOLD_2     565305
HOBBIES_2       137259
Name: sales, dtype: int64

In [174]:
data.groupby('item_id', observed=True)['sales'].sum().nlargest(10)

item_id
FOODS_3_586    268446
FOODS_3_090    213234
FOODS_3_252    157684
FOODS_3_555    129745
FOODS_3_377    104523
FOODS_3_587     91869
FOODS_3_714     83635
FOODS_3_202     75923
FOODS_3_607     72184
FOODS_3_030     67356
Name: sales, dtype: int64

In [175]:
data.groupby('state_id', observed=True)['sales'].sum().nlargest(10)

# All data is from Texas

state_id
TX    12905715
Name: sales, dtype: int64

In [176]:
data.groupby('store_id', observed=True)['sales'].sum().nlargest(10)

# All three stores are within the same order of magnitude of sales

store_id
TX_2    4808797
TX_3    4267762
TX_1    3829156
Name: sales, dtype: int64

In [4]:
prices = pd.read_parquet(f'data/prices.parquet')
prices

,store_id,item_id,sell_price
date,,,
2013-07-13,TX_1,HOBBIES_1_001,9.58
2013-07-14,TX_1,HOBBIES_1_001,9.58
2013-07-15,TX_1,HOBBIES_1_001,9.58
2013-07-16,TX_1,HOBBIES_1_001,9.58
2013-07-17,TX_1,HOBBIES_1_001,9.58
...,...,...,...
2011-02-28,TX_3,FOODS_3_825,4.00
2011-03-01,TX_3,FOODS_3_825,4.00
2011-03-02,TX_3,FOODS_3_825,4.00


In [5]:
data = data.merge(prices, on=['date', 'store_id', 'item_id'])

In [158]:
data = data.reset_index().set_index(['date', 'item_id'])

In [159]:
data['sales_dollars'] = data['sales'] * data['sell_price']

In [236]:
data.groupby('item_id', observed=True)['sales_dollars'].sum().nlargest(10)

# Two hobby items are in the top 3 of sales dollars

item_id
HOBBIES_1_354    449350.54
FOODS_3_586      438633.08
HOBBIES_1_158    349472.76
FOODS_3_202      323309.93
FOODS_3_090      294134.33
FOODS_3_252      243457.32
FOODS_3_444      237594.78
FOODS_3_555      211578.30
FOODS_3_587      200238.42
FOODS_3_120      186408.40
Name: sales_dollars, dtype: float64

In [237]:
data.groupby('store_id', observed=True)['sales_dollars'].sum().nlargest(10)

store_id
TX_2    14125923.53
TX_3    12999846.94
TX_1    11132219.10
Name: sales_dollars, dtype: float64

In [238]:
data.groupby('dept_id', observed=True)['sales_dollars'].sum().nlargest(10)

dept_id
FOODS_3        13841190.67
HOUSEHOLD_1     9221398.25
HOBBIES_1       4987195.23
FOODS_2         4740442.01
HOUSEHOLD_2     2657453.75
FOODS_1         2513567.37
HOBBIES_2        296742.29
Name: sales_dollars, dtype: float64

In [240]:
data.groupby('cat_id', observed=True)['sales_dollars'].sum().nlargest(10)

cat_id
FOODS        21095200.05
HOUSEHOLD    11878852.00
HOBBIES       5283937.52
Name: sales_dollars, dtype: float64

Before we get to modeling, let's create our evaluation setup. The models that we're going to create have a 28-day forecast horizon, and our goal is to best approximate "average" sales.

The first step is to implement our evaluation metric. The original competition used a metric called RMSSE, or "Root Mean Squared Scaled Error." It's similar to the MASE metric that we discussed before, except that the metric optimizes better for "average" sales (as opposed to MASE, which optimizes for the median, since it's an absolute error metric). The competition actually used a weighted version of RMSSE which is techincally more robust, but we're going to stick to RMSSE. Here's what RMSSE looks like:

$RMSSE = \sqrt{\frac{1}{h}\frac{\sum^{n+h}_{t=n+1} (Y_t - \hat{Y}_t)^2}{\frac{1}{n-1}\sum^n_{t=2} (Y_t - Y_{t-1})^2}}$

where $Y_t$ is the actual future value of sales at date $t$, $\hat{Y}_t$ is your forecast for date $t$, $n$ is the number of dates in our training set, and $h$ is our forecast horizon (28 days, in our case).

That looks intimidating! But, similarly to MASE, you can break it down into two parts:
- The numerator: $\frac{1}{h}\sum^{n+h}_{t=n+1} (Y_t - \hat{Y}_t)^2$, which is just the MSE for every prediction in the validation set.
- The denominator: $\frac{1}{n-1}\sum^n_{t=2} (Y_t - Y_{t-1})^2$, which is just the MSE over the entire training set if your forecast was a naive, one-day-ahead forecast. We refer to this as the "scale" since it's really just a benchmark -- errors less than this are better than the benchmark, and errors greater than this are worse.

Of course, the "naive, one-day-ahead forecast" part only works if you calculate both the numerator and denominator separately for each `id`. So, the idea here is that you are effectively calculating an RMSSE value for each `id`, and then averaging those to get the final RMSSE.

Last comment: there are products in the dataset that don't start showing sales for some time. For those products, the denominator is only supposed to be calculated after the first sale in the dataset. I'd recommend just dropping the records for those products until that first sales, which is straightforward to do using `.cumsum()` over `sales` while grouping by `id`.

In [5]:
# QUESTION: filter out products that don't have sales using cumsum

data = (
    data
    .pipe(lambda df: df[ data.groupby('id', observed=False)['sales'].cumsum() > 0 ])
)
data.shape

(10298579, 6)

Here's how you should implement your RMSSE:

1. Create a function called `rmsse` that looks like this:

`def rmsse(train, val, y_pred):`

where:
- `train` is the `pd.DataFrame` representing the training set
- `val` is the `pd.DataFrame` representing the validation set
- `y_pred` is either a `pd.Series` or `np.ndarray` that is the output of your model

2. Start by calculating the scale (i.e. denominator from above) for each `id` over the training set.

3. Then, calculate the MSE for each `id` over the validation set.

4. Merge the scale dataframe onto the dataframe that contains your validation MSE values.

5. Use the merged dataframe to calculate the RMSSE for each `id`, and finally return the average of all of those RMSSE values.

Don't worry that you haven't split your data into training and validation sets yet. I gave you a test case below to see if your code is working before you move on. Also, don't be afraid to do this in a simple, looped fashion before refactoring it into more beautiful Pandas code. Take advantage of that test case!

*Note:* If you're stuck, check the project instructions for the answer!

In [46]:
# QUESTION: implement rmsse


id
FOODS_1_001_TX_1_evaluation    3.151786
FOODS_1_001_TX_2_evaluation    2.063518
FOODS_1_001_TX_3_evaluation    1.262768
FOODS_1_002_TX_1_evaluation    0.465154
FOODS_1_002_TX_2_evaluation    0.559184
dtype: float64

In [4]:
# QUESTION: implement rmsse

def rmsse(train, val, y_pred):
    # Start by calculating the scale for each id over the training set.
    scale = train.groupby("id", observed=False).apply(lambda x: (x["sales"].diff()).dropna().pow(2).mean(), include_groups=False)

    # then, calculate the MSE for each id over the validation set
    numerator_df = pd.concat([val, pd.DataFrame(y_pred, columns=["pred"])], axis=1).set_index("id")
    mse = (numerator_df['sales'] - numerator_df['pred']) ** 2
    mse = mse.groupby("id").mean()

    # Calculate the RMSSE for each id, and finally return the average of all of those RMSSE values.
    return np.sqrt((mse / scale).groupby("id").sum()).mean()

In [22]:
# Alternative solution:

def rmsse(train, val, y_pred):
    train_scale = (
        train
        .assign(
            scale=train.groupby('id').sales.diff() ** 2,
        )
        .groupby('id')
        .scale
        .mean()
    )
    
    score = (
        val
        .assign(squared_error=(val.sales - y_pred) ** 2)
        .groupby('id')
        .squared_error
        .mean()
        .to_frame()
        .merge(train_scale, on='id')
        .assign(rmsse=lambda x: np.sqrt(x.squared_error / x.scale))
        .rmsse
        .mean()
    )

    return score

In [23]:
def test_rmsse():
    test_train = pd.DataFrame({
        'id': ['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'],
        'sales': [3, 2, 5, 100, 150, 60, 10, 20, 30],
    })
    test_val = pd.DataFrame({
        'id': ['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'],
        'sales': [6, 1, 4, 200, 120, 270, 10, 20, 30],
    })
    test_y_pred = pd.Series([1, 2, 3, 180, 160, 240, 20, 30, 40])
    assert np.abs(rmsse(test_train, test_val, test_y_pred) - 0.92290404515501) < 1e-6

test_rmsse()

# Fitting models

Now it's time to fit a LightGBM model. You'll be fitting a single model that predicts 28 days into the future. Remember to focus less on hyperparameter tuning and a lot more on the data and your features.

Feature engineering can be tricky! I'd recommend giving it a shot yourself, but if you get stuck, remember that I provided code to calculate rolling and lagged features in the course modules.

Remember to also use the external datasets that I provided, they'll be very useful!

Here's some sample feature engineering code to get started with:

In [208]:
calendar = pd.read_parquet(f'data/calendar.parquet')
calendar

,snap_TX,event_name_1,event_type_1,event_name_2,event_type_2
date,,,,,
2011-01-29,0,None,None,None,None
2011-01-30,0,None,None,None,None
2011-01-31,0,None,None,None,None
2011-02-01,1,None,None,None,None
2011-02-02,0,None,None,None,None
...,...,...,...,...,...
2016-06-15,1,None,None,None,None
2016-06-16,0,None,None,None,None
2016-06-17,0,None,None,None,None


In [170]:
data

dept_id cat_id store_id state_id  sales  sell_price  \
date       item_id                                                            
2013-01-01 FOODS_1_004  FOODS_1  FOODS     TX_1       TX     20        1.78   
           FOODS_1_004  FOODS_1  FOODS     TX_2       TX     20        1.78   
           FOODS_1_004  FOODS_1  FOODS     TX_3       TX      4        1.78   
           FOODS_1_005  FOODS_1  FOODS     TX_2       TX      1        3.28   
           FOODS_1_009  FOODS_1  FOODS     TX_2       TX      3        2.68   
...                         ...    ...      ...      ...    ...         ...   
2016-05-22 FOODS_2_117  FOODS_2  FOODS     TX_1       TX      0        5.97   
           FOODS_2_256  FOODS_2  FOODS     TX_3       TX      1        6.36   
           FOODS_2_256  FOODS_2  FOODS     TX_1       TX      0        6.36   
           FOODS_3_296  FOODS_3  FOODS     TX_2       TX      3        2.00   
           FOODS_2_069  FOODS_2  FOODS     TX_3       TX      0        8.98   

                        sales_dollars  
date       item_id                     
2013-01-01 FOODS_1_004          35.60  
           FOODS_1_004          35.60  
           FOODS_1_004           7.12  
           FOODS_1_005           3.28  
           FOODS_1_009           8.04  
...                               ...  
2016-05-22 FOODS_2_117           0.00  
           FOODS_2_256           6.36  
           FOODS_2_256           0.00  
           FOODS_3_296           6.00  
           FOODS_2_069           0.00  

[10455914 rows x 7 columns]

In [7]:
class LagFeature:
    def __init__(self, shift_length, forecast_horizon):
        self.shift_length = shift_length
        self.forecast_horizon = forecast_horizon
        
    @property
    def feature_name(self):
        return f'lag_{self.shift_length}_{self.forecast_horizon}'
    
    def calc_lag(self, series):
        return (
            series
            .groupby('id', observed=True)
            .shift(self.forecast_horizon + self.shift_length)
            .rename(self.feature_name)
        )

In [8]:
class RollingAggFeature:
    def __init__(self, window_length, forecast_horizon, agg_func='mean', by_day_of_week=False):
        self.window_length = window_length
        self.forecast_horizon = forecast_horizon
        self.agg_func = agg_func
        self.by_day_of_week = by_day_of_week
    
    @property
    def feature_name(self):
        if self.by_day_of_week:
            return f'by_week_rolling_{self.agg_func}_{self.window_length}_{self.forecast_horizon}'
        else:
            return f'rolling_{self.agg_func}_{self.window_length}_{self.forecast_horizon}'
    
    def calc_rolling_agg(self, series):
        group_cols = ['id']
        if self.by_day_of_week:
            group_cols += ['day_of_week']

        return (
            series
            .to_frame()
            .assign(day_of_week=series.index.get_level_values('date').dayofweek)
            .groupby('id', observed=True)
            .rolling(self.window_length, closed='right', min_periods=1) # only requires 1 observation to be non-NaN
            .agg({'sales': self.agg_func})
            .droplevel(-1)
            .reset_index()
            .assign(date=lambda x: x.date + pd.Timedelta(days=28))
            .set_index(['date', 'id'])
            .rename(columns={'sales': self.feature_name})
        )

In [9]:
def calc_diff(series):    
    return (
        series
        .groupby('id', observed=True)
        .diff()
        .rename(f'{series.name}_diff')
    )

In [18]:
def feature_engineering(df):
    df_with_added_features = df.copy(deep=True)
    forecast_horizon = 28
    forecast_horizon_seasonal = 4
    
    features = ['sales']
    
    for lag in [1, 7, 365]:
        lag = LagFeature(lag, forecast_horizon)
        df_with_added_features[lag.feature_name] = lag.calc_lag(df['sales'])
        features.append(lag.feature_name)

    for window in [28, 365]:
        roll_feature = RollingAggFeature(window, forecast_horizon)
        df_with_added_features[roll_feature.feature_name] = roll_feature.calc_rolling_agg(df['sales'])
        features.append(roll_feature.feature_name)

    for window in [4, 52]:
        roll_feature = RollingAggFeature(window, forecast_horizon_seasonal, by_day_of_week=True)
        df_with_added_features[roll_feature.feature_name] = roll_feature.calc_rolling_agg(df['sales'])
        features.append(roll_feature.feature_name)

    for diff in ['sales']:
        diff_series = calc_diff(df_with_added_features[diff])
        df_with_added_features[diff_series.name] = diff_series
        features.append(diff_series.name)

    train = df_with_added_features.loc[:"2015-06-01", :]
    val = df_with_added_features.loc["2015-06-01":, :]
    
    return train, val, features


In [19]:
train, val, features = feature_engineering(data)

In [20]:
train.dropna()

item_id  dept_id cat_id store_id  \
date       id                                                                  
2014-01-29 FOODS_1_004_TX_1_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_1   
           FOODS_1_004_TX_2_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_2   
           FOODS_1_004_TX_3_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_3   
           FOODS_1_005_TX_2_evaluation  FOODS_1_005  FOODS_1  FOODS     TX_2   
           FOODS_1_009_TX_2_evaluation  FOODS_1_009  FOODS_1  FOODS     TX_2   
...                                             ...      ...    ...      ...   
2015-06-01 FOODS_2_117_TX_1_evaluation  FOODS_2_117  FOODS_2  FOODS     TX_1   
           FOODS_2_256_TX_3_evaluation  FOODS_2_256  FOODS_2  FOODS     TX_3   
           FOODS_2_256_TX_1_evaluation  FOODS_2_256  FOODS_2  FOODS     TX_1   
           FOODS_3_296_TX_2_evaluation  FOODS_3_296  FOODS_3  FOODS     TX_2   
           FOODS_2_069_TX_3_evaluation  FOODS_2_069  FOODS_2  FOODS     TX_3   

                                       state_id  sales  lag_1_28  lag_7_28  \
date       id                                                                
2014-01-29 FOODS_1_004_TX_1_evaluation       TX      8       6.0       0.0   
           FOODS_1_004_TX_2_evaluation       TX      5      13.0       0.0   
           FOODS_1_004_TX_3_evaluation       TX      8       7.0       0.0   
           FOODS_1_005_TX_2_evaluation       TX      0       2.0       0.0   
           FOODS_1_009_TX_2_evaluation       TX      0       0.0       0.0   
...                                         ...    ...       ...       ...   
2015-06-01 FOODS_2_117_TX_1_evaluation       TX      0       0.0       0.0   
           FOODS_2_256_TX_3_evaluation       TX      0       0.0       0.0   
           FOODS_2_256_TX_1_evaluation       TX      0       0.0       0.0   
           FOODS_3_296_TX_2_evaluation       TX      0       0.0       0.0   
           FOODS_2_069_TX_3_evaluation       TX      0       0.0       0.0   

                                        lag_365_28  rolling_mean_28_28  \
date       id                                                            
2014-01-29 FOODS_1_004_TX_1_evaluation        20.0            7.321429   
           FOODS_1_004_TX_2_evaluation        20.0           12.071429   
           FOODS_1_004_TX_3_evaluation         4.0            9.642857   
           FOODS_1_005_TX_2_evaluation         1.0            2.428571   
           FOODS_1_009_TX_2_evaluation         3.0            0.000000   
...                                            ...                 ...   
2015-06-01 FOODS_2_117_TX_1_evaluation         0.0            0.000000   
           FOODS_2_256_TX_3_evaluation         0.0            0.000000   
           FOODS_2_256_TX_1_evaluation         0.0            0.000000   
           FOODS_3_296_TX_2_evaluation         0.0            0.000000   
           FOODS_2_069_TX_3_evaluation         0.0            0.000000   

                                        rolling_mean_365_28  \
date       id                                                 
2014-01-29 FOODS_1_004_TX_1_evaluation             7.328767   
           FOODS_1_004_TX_2_evaluation            13.024658   
           FOODS_1_004_TX_3_evaluation             7.276712   
           FOODS_1_005_TX_2_evaluation             1.115068   
           FOODS_1_009_TX_2_evaluation             0.276712   
...                                                     ...   
2015-06-01 FOODS_2_117_TX_1_evaluation             0.000000   
           FOODS_2_256_TX_3_evaluation             0.000000   
           FOODS_2_256_TX_1_evaluation             0.000000   
           FOODS_3_296_TX_2_evaluation             0.000000   
           FOODS_2_069_TX_3_evaluation             0.000000   

                                        by_week_rolling_mean_4_4  \
date       id                                                      
2014-01-29 FOODS_1_004_TX_1_evaluation                      2.75   
 

And some sample model fitting code:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb

# some parameters to play around with. Not the best by any means!
params = dict(
    objective='tweedie',
    tweedie_variance_power=1.1,
    learning_rate=0.05,
    min_samples_leaf=100,
    subsample=0.3,
    feature_fraction=0.3,
    deterministic=True,
)

train, val, features = feature_engineering(data)

train_dset = lgb.Dataset(
    train[features], 
    train['sales'],
)

val_dset = lgb.Dataset(
    val[features], 
    val['sales'],
)

callbacks = [
    lgb.early_stopping(100),
    lgb.log_evaluation(50)
]

model = lgb.train(
    params, 
    train_dset,
    num_boost_round=1000,
    valid_sets=[val_dset],
    callbacks=callbacks,
)

In [ ]:
preds = model.predict(val[features])

In [27]:
val

item_id  dept_id cat_id store_id  \
date       id                                                                  
2015-06-01 FOODS_1_004_TX_1_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_1   
           FOODS_1_004_TX_2_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_2   
           FOODS_1_004_TX_3_evaluation  FOODS_1_004  FOODS_1  FOODS     TX_3   
           FOODS_1_005_TX_2_evaluation  FOODS_1_005  FOODS_1  FOODS     TX_2   
           FOODS_1_009_TX_2_evaluation  FOODS_1_009  FOODS_1  FOODS     TX_2   
...                                             ...      ...    ...      ...   
2016-05-22 FOODS_2_117_TX_1_evaluation  FOODS_2_117  FOODS_2  FOODS     TX_1   
           FOODS_2_256_TX_3_evaluation  FOODS_2_256  FOODS_2  FOODS     TX_3   
           FOODS_2_256_TX_1_evaluation  FOODS_2_256  FOODS_2  FOODS     TX_1   
           FOODS_3_296_TX_2_evaluation  FOODS_3_296  FOODS_3  FOODS     TX_2   
           FOODS_2_069_TX_3_evaluation  FOODS_2_069  FOODS_2  FOODS     TX_3   

                                       state_id  sales  lag_1_28  lag_7_28  \
date       id                                                                
2015-06-01 FOODS_1_004_TX_1_evaluation       TX      7      11.0      15.0   
           FOODS_1_004_TX_2_evaluation       TX      2       5.0       3.0   
           FOODS_1_004_TX_3_evaluation       TX     25      20.0      21.0   
           FOODS_1_005_TX_2_evaluation       TX      2       1.0       1.0   
           FOODS_1_009_TX_2_evaluation       TX      0       1.0       1.0   
...                                         ...    ...       ...       ...   
2016-05-22 FOODS_2_117_TX_1_evaluation       TX      0       1.0       0.0   
           FOODS_2_256_TX_3_evaluation       TX      1       0.0       3.0   
           FOODS_2_256_TX_1_evaluation       TX      0       0.0       1.0   
           FOODS_3_296_TX_2_evaluation       TX      3       2.0       3.0   
           FOODS_2_069_TX_3_evaluation       TX      0       0.0       0.0   

                                        lag_365_28  rolling_mean_28_28  \
date       id                                                            
2015-06-01 FOODS_1_004_TX_1_evaluation         9.0            9.035714   
           FOODS_1_004_TX_2_evaluation        11.0            3.892857   
           FOODS_1_004_TX_3_evaluation        12.0           16.428571   
           FOODS_1_005_TX_2_evaluation         0.0            0.821429   
           FOODS_1_009_TX_2_evaluation         0.0            0.857143   
...                                            ...                 ...   
2016-05-22 FOODS_2_117_TX_1_evaluation         0.0            0.750000   
           FOODS_2_256_TX_3_evaluation         0.0            1.071429   
           FOODS_2_256_TX_1_evaluation         0.0            0.464286   
           FOODS_3_296_TX_2_evaluation         0.0            1.285714   
           FOODS_2_069_TX_3_evaluation         0.0            0.071429   

                                        rolling_mean_365_28  \
date       id                                                 
2015-06-01 FOODS_1_004_TX_1_evaluation             5.750685   
           FOODS_1_004_TX_2_evaluation             4.652055   
           FOODS_1_004_TX_3_evaluation             9.747945   
           FOODS_1_005_TX_2_evaluation             1.158904   
           FOODS_1_009_TX_2_evaluation             0.394521   
...                                                     ...   
2016-05-22 FOODS_2_117_TX_1_evaluation             0.287671   
           FOODS_2_256_TX_3_evaluation             0.345205   
           FOODS_2_256_TX_1_evaluation             0.158904   
           FOODS_3_296_TX_2_evaluation             0.309589   
           FOODS_2_069_TX_3_evaluation             0.005479   

                                        by_week_rolling_mean_4_4  \
date       id                                                      
2015-06-01 FOODS_1_004_TX_1_evaluation                     11.00   
 

In [29]:
print(f'RMSSE: {rmsse(train, val, preds)}')

/tmp/ipykernel_132282/3521453703.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  scale=train.groupby('id').sales.diff() ** 2,
/tmp/ipykernel_132282/3521453703.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('id')


RMSSE: inf


/tmp/ipykernel_132282/3521453703.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('id')


Write a brief summary of what helped your models and what didn't help. Was it what you expected?

I'm finishing this late and I just need to get it in. Sorry for the lack of insight!